#### Imports - Crisis Prolific Data Analysis

In [1]:
import os
import sys

In [2]:
import numpy
import pandas

In [3]:
import spacy

In [4]:
import  contractions

In [5]:
import spellchecker

In [6]:
import missingno

In [7]:
import sklearn

In [8]:
import sklearn.cluster
import sklearn.metrics
import sklearn.ensemble
import sklearn.model_selection

In [9]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import FeatureAgglomeration

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import silhouette_score

#### Reading Data - We read all the files for Crisis Prolific Survey and add columns for adult and parent demographic
#### Additionally, add the year and month as well to normalize data. This data is concatenated together to form one consolidated dataframe

In [13]:
crisis_pa_adlt_202004 = pandas.read_csv("../../Data/ProlificAcademic/April 2020/Data/CRISIS_Adult_April_2020.csv")
crisis_pa_prnt_202004 = pandas.read_csv("../../Data/ProlificAcademic/April 2020/Data/CRISIS_Parent_April_2020.csv")
crisis_pa_adlt_202004["ind_adlt"] = 1
crisis_pa_adlt_202004["ind_prnt"] = 0
crisis_pa_adlt_202004["num_yr"] = 2020
crisis_pa_adlt_202004["num_mnth"] = 4
crisis_pa_prnt_202004["ind_adlt"] = 0
crisis_pa_prnt_202004["ind_prnt"] = 1
crisis_pa_prnt_202004["num_yr"] = 2020
crisis_pa_prnt_202004["num_mnth"] = 4

In [14]:
crisis_pa_adlt_202005 = pandas.read_csv("../../Data/ProlificAcademic/May 2020/Data/CRISIS_Adult_May_2020.csv")
crisis_pa_prnt_202005 = pandas.read_csv("../../Data/ProlificAcademic/May 2020/Data/CRISIS_Parent_May_2020.csv")
crisis_pa_adlt_202005["ind_adlt"] = 1
crisis_pa_adlt_202005["ind_prnt"] = 0
crisis_pa_adlt_202005["num_yr"] = 2020
crisis_pa_adlt_202005["num_mnth"] = 5
crisis_pa_prnt_202005["ind_adlt"] = 0
crisis_pa_prnt_202005["ind_prnt"] = 1
crisis_pa_prnt_202005["num_yr"] = 2020
crisis_pa_prnt_202005["num_mnth"] = 5

In [15]:
crisis_pa_adlt_202011 = pandas.read_csv("../../Data/ProlificAcademic/November 2020/Data/CRISIS_Adult_November_2020.csv", low_memory=False)
crisis_pa_prnt_202011 = pandas.read_csv("../../Data/ProlificAcademic/November 2020/Data/CRISIS_Parent_November_2020.csv", low_memory=False)
crisis_pa_adlt_202011["ind_adlt"] = 1
crisis_pa_adlt_202011["ind_prnt"] = 0
crisis_pa_adlt_202011["num_yr"] = 2020
crisis_pa_adlt_202011["num_mnth"] = 11
crisis_pa_prnt_202011["ind_adlt"] = 0
crisis_pa_prnt_202011["ind_prnt"] = 1
crisis_pa_prnt_202011["num_yr"] = 2020
crisis_pa_prnt_202011["num_mnth"] = 11

In [16]:
crisis_pa_adlt_202104 = pandas.read_csv("../../Data/ProlificAcademic/April 2021/Data/CRISIS_Adult_April_2021.csv", low_memory=False)
crisis_pa_prnt_202104 = pandas.read_csv("../../Data/ProlificAcademic/April 2021/Data/CRISIS_Parent_April_2021.csv", low_memory=False)
crisis_pa_adlt_202104["ind_adlt"] = 1
crisis_pa_adlt_202104["ind_prnt"] = 0
crisis_pa_adlt_202104["num_yr"] = 2021
crisis_pa_adlt_202104["num_mnth"] = 4
crisis_pa_prnt_202104["ind_adlt"] = 0
crisis_pa_prnt_202104["ind_prnt"] = 1
crisis_pa_prnt_202104["num_yr"] = 2021
crisis_pa_prnt_202104["num_mnth"] = 4

In [17]:
crisis_pa_adlt = pandas.concat([crisis_pa_adlt_202004, crisis_pa_adlt_202005, crisis_pa_adlt_202011, crisis_pa_adlt_202104])

In [18]:
crisis_pa_prnt = pandas.concat([crisis_pa_prnt_202004, crisis_pa_prnt_202005, crisis_pa_prnt_202011, crisis_pa_prnt_202104])

In [19]:
crisis_pa_adlt.describe()

,country,age,sex,raceethnicity___1,raceethnicity___2,raceethnicity___3,raceethnicity___4,raceethnicity___5,raceethnicity___6,raceethnicity___7,...,iat_12,iat_13,iat_14,iat_15,iat_16,iat_17,iat_18,iat_19,iat_20,internet_addiction_test_iat_complete
count,10351.000000,10340.000000,2839.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,...,2038.000000,2038.000000,2037.000000,2040.000000,2042.000000,2039.000000,2040.000000,2040.000000,2037.000000,2125.000000
mean,186.342189,38.956383,1.579429,0.677274,0.022392,0.020821,0.005303,0.017285,0.047338,0.103712,...,1.371443,0.792934,1.208640,0.726471,1.526934,1.299657,0.677941,0.912745,0.696613,1.923765
std,3.335535,14.746920,0.501526,0.467565,0.147971,0.142799,0.072639,0.130346,0.212383,0.304917,...,1.238758,0.858301,1.179043,0.904832,1.303337,1.160756,0.893387,1.082762,0.886071,0.383051
min,17.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,186.000000,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,186.000000,36.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000
75%,187.000000,51.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000
max,187.000000,83.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2.000000


#### Create a dictionary to identify missing values in the Crisis Prolific Survey

In [20]:
# Initialize a dictionary to add data
missing_values_dict = {}
missing_values_dict["nme_col"] = []
missing_values_dict["cnt_missing"] = []
missing_values_dict["cnt_avl"] = []
# Print the percentage of non available values across each column.
for c in crisis_pa_adlt.columns:
    missing_values_dict["nme_col"].append(c)
    missing_values_dict["cnt_missing"].append(len(crisis_pa_adlt[pandas.isna(crisis_pa_adlt[c])]))
    missing_values_dict["cnt_avl"].append(len(crisis_pa_adlt[~pandas.isna(crisis_pa_adlt[c])]))
# Missing values dataframe.
crisis_pa_adlt_msng = pandas.DataFrame(missing_values_dict)

#### The Tall and Weight columns have been provided in different measurements. The purpose of code below is to rationalize and convert to same measurement system
#### Additionally, we've observed outliers in data and based on percentile filtering, have filtered out the same. We use 10th to 95th Percentile band in "Tall" and 5th to 95th Percentile band in "Weight"


In [21]:
# Rationalize tall / height
crisis_pa_adlt[crisis_pa_adlt["measurementheight"]==1][["tall", "measurementheight"]]
# If height is in inches, convert it to cm
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt["tall"]
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt[["tall", "measurementheight"]].apply(lambda r: r[0] if r[1]==2 else r[0] * 2.54, axis=1)
# 10th and 95th percentile height
tall_cm_095 = crisis_pa_adlt["tall_cm"].quantile(0.95)
tall_cm_010 = crisis_pa_adlt["tall_cm"].quantile(0.10)
# Filter outliers
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt["tall_cm"].apply(lambda r: r if (r>=tall_cm_010 and r<=tall_cm_095) else numpy.nan)

In [22]:
# Rationalize weight
crisis_pa_adlt[crisis_pa_adlt["measurementweight"]==1][["weight", "measurementweight"]]
# If weight is in lbs, convert it to kg
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt["weight"]
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt[["weight", "measurementweight"]].apply(lambda r: r[0] if r[1]==2 else r[0] * 0.453, axis=1)
# Percentiles
weight_kg_005 = crisis_pa_adlt["weight_kg"].quantile(0.05)
weight_kg_095 = crisis_pa_adlt["weight_kg"].quantile(0.95)
# Filter outliers
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt["weight_kg"].apply(lambda r: r if (r>=weight_kg_005 and r<=weight_kg_095) else numpy.nan)

#### Approach - Segregate columns / features into different Demographic levels. Based on health indicators, substance use etc.

In [23]:
# Listing columns that are focused on health demographics
# Includes - Age, Gender, Height, Weight, Health Conditions, Hours of Sleep, Quality, Trouble
dmg_hlth_cols = [
    "age",
    "gender",
    "tall_cm",
    "weight_kg",
    "cigarettesyn",
    "healthconditions___1",
    "healthconditions___2",
    "healthconditions___3",
    "healthconditions___4",
    "healthconditions___5",
    "healthconditions___6",
    "healthconditions___7",
    "healthconditions___8",
    "healthconditions___9",
    "healthconditions___10",
    "healthconditions___11",
    "healthconditions___12",
    "healthconditions___13",
    "healthconditions___14",
    "healthconditions___15",
    "healthconditions___16",
    "healthconditions___17",
    "healthconditions___18",
    "healthconditions___19",
    "healthconditions___20",
    "working___1",
    "working___2",
    "working___3",
    "working___4",
    "working___5",
    "working___6",
    "working___7",
    "working___8",
    "location",
    "householdnumber",
    "householdcomp___1",
    "householdcomp___2",
    "householdcomp___3",
    "householdcomp___4",
    "householdcomp___5",
    "householdcomp___6",
    "householdcomp___7",
    "hoursofsleepweekdays",
    "hoursofsleepweekends",
    "troublesleeping",
    "sleepquality",
    "timeoutside",
    "readingtalking"
]


In [24]:
# Columns focusing on substance and medicine usage
dmg_subs_cols = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine",
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother",
    "substanceproblems",
    "substancecomplaint"
]

In [25]:
# Considering substance focused columns minus - problems and complaints 
dmg_subs_cols_imp_1 = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine",
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother"
] 

In [26]:
# Created 2 segregations - 1 for substance and 1 for meds
dmg_subs_cols_imp_1_sub = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine"
]
dmg_subs_cols_imp_1_med = [
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother"
]

In [27]:
# Creating a dictionary to map substance use to frequency
# 1, Not at all | 2, Rarely | 3, Once a month | 4, Several times a month | 5, Once a week | 6, Several times a week | 7, Once a day | 8, More than once a day
subs_freq_dict = {
    1: 0,
    2: 0.01,
    3: 0.03,
    4: 0.075,
    5: 0.15,
    6: 0.33,
    7: 1,
    8: 3
}

#### Imputing dataset for substance related columns. Also we created a new column _imp_freq that provides the numeric frequency instead of a categorical value

In [28]:

for col in dmg_subs_cols_imp_1:
    # First apply 1 in cases where response isn't provided
    crisis_pa_adlt[col + "_imp"] = crisis_pa_adlt[col].apply(lambda r: 1.0 if numpy.isnan(r) else r)
    crisis_pa_adlt[col + "_imp"] = crisis_pa_adlt[col + "_imp"].astype(int)
    crisis_pa_adlt[col + "_imp_freq"] = crisis_pa_adlt[col + "_imp"].apply(lambda r: subs_freq_dict[r])

In [29]:
# Other columns in substances
crisis_pa_adlt["substanceproblems_imp"] = crisis_pa_adlt["substanceproblems"].apply(lambda r: 0 if numpy.isnan(r) else r).astype(int)
crisis_pa_adlt["substancecomplaint_imp"] = crisis_pa_adlt["substancecomplaint"].apply(lambda r: 0 if numpy.isnan(r) else r).astype(int) 

#### Rationalize column names across different monthly level concatenated feeds

In [30]:
# Create a common column for worry. The idea is to take average and maximum worry of an individual across all
crisis_pa_adlt["max_worried"] = crisis_pa_adlt[["worriedyourself", "worriedothers", "worriedmental", "worriedphysical"]].apply(lambda r: numpy.max(r), axis=1)
crisis_pa_adlt["avg_worried"] = crisis_pa_adlt[["worriedyourself", "worriedothers", "worriedmental", "worriedphysical"]].apply(lambda r: numpy.average(r), axis=1)
crisis_pa_adlt["ind_worried"] = crisis_pa_adlt["max_worried"].apply(lambda r: 1 if r > 1 else 0)

#### Trying Agglomerative and K-means clustering to identify common groups of individuals

In [31]:
# Agglomerative clustering based on subs (all) columns (imputed)
subs_agc = AgglomerativeClustering(n_clusters=8, affinity="euclidean", linkage="ward")
# Data to be taken for months of May 2020, November 2020 and April 2021. These columns are not present in April 2020
crisis_pa_adlt_f = crisis_pa_adlt[((crisis_pa_adlt["num_yr"]==2020) & (crisis_pa_adlt["num_mnth"].isin([5, 11]))) | (crisis_pa_adlt["num_yr"]==2021)].copy(True)
crisis_pa_adlt_f["subs_imp_med_agc_1"] = subs_agc.fit_predict(crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1]])
# Clustering evaluation using Silhouette score
# Post iterative experimentation, the clustering silhouette score ~ 0.82
print(silhouette_score(crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1]], crisis_pa_adlt_f["subs_imp_med_agc_1"]))
# Cluster level distribution
crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1] + ["subs_imp_med_agc_1"]].groupby(["subs_imp_med_agc_1"]).agg(["count", "mean", "median", "sum"])
# Observations
# There is one key cluster (1) [Count - 5185] that has least values within folks who've reported any substance consumption
# Other clusters are segregated based on any one or more type of substance / medicine usage
# Cluster 5 has high level of alcohol consumption (Mean - 5), with relatively lower level of other substances

# Cluster level distribution - of columns - worries
# There is NO observed relationship between substance / medicine intake with level of worries as provided in the response
# While there is a slight fluctuation across clusters, there is no definite relationship dictating that the individuals more prone to substance and medicine intake are more / less likely to be worried
crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1] + ["subs_imp_med_agc_1", "max_worried", "avg_worried", "ind_worried"]].groupby(["subs_imp_med_agc_1"]).agg(["count", "mean", "median", "sum"])

# Check if there is any correlation of resulting clusters with age
# Key observations - 
# Clusters with moderate alcohol and high marijuana intake correlates with low mean and median age (30)
# Clusters with moderate alcohol and high cigarette intage correlates with high mean / median age. They also have a relatively higher frequency of medicine intake (44) 
crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1] + ["subs_imp_med_agc_1", "age"]].groupby(["subs_imp_med_agc_1"]).agg(["count", "mean", "median", "sum"])

0.763570089532019


alcohol_imp                        cigarettes_imp  \
                         count      mean median   sum          count   
subs_imp_med_agc_1                                                     
0                           81  4.345679    5.0   352             81   
1                         5185  1.176278    1.0  6099           5185   
2                          292  3.582192    3.0  1046            292   
3                          222  3.387387    3.0   752            222   
4                          184  4.032609    4.0   742            184   
5                         1413  5.417551    6.0  7655           1413   
6                           41  3.853659    4.0   158             41   
7                           78  3.974359    4.0   310             78   

                                          vape_imp            ...  \
                        mean median   sum    count      mean  ...   
subs_imp_med_agc_1                                            ...   
0                   7.172840    8.0   581       81  3.913580  ...   
1                   1.008293    1.0  5228     5185  1.004822  ...   
2                   7.520548    8.0  2196      292  1.222603  ...   
3                   1.306306    1.0   290      222  7.454955  ...   
4                   1.125000    1.0   207      184  1.146739  ...   
5                   1.130219    1.0  1597     1413  1.090587  ...   
6                   4.341463    6.0   178       41  1.512195  ...   
7                   6.717949    7.0   524       78  6.987179  ...   

                   rxstimulants_imp       rxother_imp                         \
                             median   sum       count      mean median   sum   
subs_imp_med_agc_1                                                             
0                               1.0   193          81  1.950617    1.0   158   
1                               1.0  5193        5185  1.003472    1.0  5203   
2                               1.0   303         292  1.123288    1.0   328   
3                               1.0   240         222  1.202703    1.0   267   
4                               1.0   188         184  1.059783    1.0   195   
5                               1.0  1423        1413  1.087049    1.0  1536   
6                               1.0    41          41  1.024390    1.0    42   
7                               1.0    82          78  1.051282    1.0    82   

                     age                              
                   count       mean median       sum  
subs_imp_med_agc_1                                    
0                     81  38.012346   34.0    3079.0  
1                   5166  38.242354   35.0  197560.0  
2                    292  44.034247   42.0   12858.0  
3                    222  39.225225   37.0    8708.0  
4                    184  32.820652   30.0    6039.0  
5                   1409  42.765082   39.0   60256.0  
6                     41  41.317073   38.0    1694.0  
7                     78  41.384615   39.5    3228.0  

[8 rows x 48 columns]

#### Note - We tried experimenting with K-Means based clustering as well and did not find any evidence / strong correlation therein

In [32]:
# Confirming via the Correlation matrix that there seems to be very less / no correlation between worries and substance / medicine use
crisis_pa_adlt_f[[col + "_imp" for col in dmg_subs_cols_imp_1] + ["subs_imp_med_agc_1", "max_worried", "avg_worried", "ind_worried", "age"]].corr(method="spearman")

,alcohol_imp,cigarettes_imp,vape_imp,tobacco_imp,marijuana_imp,heroin_imp,cocaine_imp,rxpain_imp,rxsleeping_imp,rxstimulants_imp,rxother_imp,subs_imp_med_agc_1,max_worried,avg_worried,ind_worried,age
alcohol_imp,1.000000,0.266604,0.215066,0.161966,0.273270,0.065997,0.149122,0.145498,0.098766,0.080052,0.104004,0.768388,-0.118642,-0.127082,0.012016,0.100920
cigarettes_imp,0.266604,1.000000,0.382954,0.292363,0.311520,0.178149,0.265350,0.151510,0.142131,0.173373,0.143556,0.235790,0.007345,0.001133,0.023253,0.005843
vape_imp,0.215066,0.382954,1.000000,0.276114,0.298276,0.195270,0.184516,0.180519,0.124888,0.216333,0.176254,0.248479,-0.017108,-0.013066,0.008870,-0.024075
tobacco_imp,0.161966,0.292363,0.276114,1.000000,0.246642,0.269194,0.208294,0.140820,0.168912,0.250086,0.159930,0.148851,-0.005453,-0.010385,0.006771,-0.019931
marijuana_imp,0.273270,0.311520,0.298276,0.246642,1.000000,0.159418,0.293862,0.130288,0.097125,0.198943,0.092526,0.159493,0.021025,0.004991,0.031153,-0.118879
heroin_imp,0.065997,0.178149,0.195270,0.269194,0.159418,1.000000,0.392943,0.254191,0.164501,0.405751,0.325980,-0.045786,-0.003224,-0.006587,0.006102,-0.006062
cocaine_imp,0.149122,0.265350,0.184516,0.208294,0.293862,0.392943,1.000000,0.185088,0.130145,0.263082,0.195255,0.082239,-0.003372,-0.010889,0.003393,-0.046611
rxpain_imp,0.145498,0.151510,0.180519,0.140820,0.130288,0.254191,0.185088,1.000000,0.173998,0.201885,0.255754,0.126158,0.040517,0.041299,0.028966,0.054298
rxsleeping_imp,0.098766,0.142131,0.124888,0.168912,0.097125,0.164501,0.130145,0.173998,1.000000,0.229878,0.184239,0.083934,0.044951,0.050590,0.026596,0.002136
rxstimulants_imp,0.080052,0.173373,0.216333,0.250086,0.198943,0.405751,0.263082,0.201885,0.229878,1.000000,0.280878,-0.034238,0.006047,0.014176,0.011475,-0.015423


#### Also tried to fit a model to see if there is any predictive possibility w.r.t predicting whether an individual is more prone to be worried

In [33]:
# Filter data for cases where mental health is available.
crisis_pa_adlt_wr = crisis_pa_adlt_f.copy(True)
crisis_pa_adlt_wr["max_worried"] = crisis_pa_adlt_wr["max_worried"].apply(lambda r: 1 if numpy.isnan(r) else r)
crisis_pa_adlt_wr["max_worried"] = crisis_pa_adlt_wr["max_worried"].astype(int)
crisis_pa_adlt_wr["max_worried_1"] = crisis_pa_adlt_wr["max_worried"].apply(lambda r: 0 if r in (1, 2, 3) else 1)

#### We did not observe (through modeling as well) any predictive possibility from Substance -> Worried

In [34]:
for v in crisis_pa_adlt_wr["max_worried_1"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_wr_1 = crisis_pa_adlt_wr.copy(True)
    crisis_pa_adlt_wr_1["wr_tgt"] = crisis_pa_adlt_wr_1["max_worried_1"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Maximum worried level - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_wr_1[[col + "_imp" for col in dmg_subs_cols_imp_1]], crisis_pa_adlt_wr_1["wr_tgt"], test_size=0.2, random_state=100)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.05, n_estimators=1000, max_depth=5)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Maximum worried level - 1
INFO: Training Metrics
[[3397   44]
 [2238  317]]
INFO: Testing metrics
[[770  52]
 [630  48]]
0.503768059772194

INFO: Training model for Maximum worried level - 0
INFO: Training Metrics
[[ 312 2243]
 [  40 3401]]
INFO: Testing metrics
[[ 48 630]
 [ 54 768]]
0.5025515147600284



#### Shifting Focus on Health specific Demographics
#### Observations from Data cleaning
#### Checking the availability of data as follows - 
##### crisis_pa_adlt[["num_yr", "num_mnth", "gender"]].groupby(["num_yr", "num_mnth"]).count()
#### Column - age is present in all months, tall is not present in 2020-05, gender is not present in 2020-04 and 2020-05
#### Column - CigaretteSyn is present only in 2020-11
#### Column - HealthConditions___X are not present in 2020-05
#### Column - troublesleeping and sleep quality present only in 2020-11 and 2021-04
#### Column - Working___1 to 8 not present only in May, rest not present in April and May 2020
#### Column - School___X only present in April 2021 survey
#### Column - Location - Not present in May 2020


In [38]:
crisis_pa_adlt[["num_yr", "num_mnth", "physicalhealth"]].groupby(["num_yr", "num_mnth"]).count()

physicalhealth
num_yr num_mnth                
2020   4                   2860
       5                      0
       11                  2221
2021   4                      0

In [39]:
# List of columns with availability more than 2
cols_gt_2 = list()
# Year and month with highest availability
yr_mnth_avl = {}

# The purpose of this loop is to iterate over selected cols and find out the ones which are most readily available, as well as the year and month combinations that are well populated
for col in dmg_hlth_cols:
    dat_avl = crisis_pa_adlt[["num_yr", "num_mnth", col]].groupby(["num_yr", "num_mnth"]).count()
    dat_avl = dat_avl.reset_index()
    # Iterate over rows
    for row in dat_avl.iterrows():
        # Create year-month key
        ym_key = str(row[1]["num_yr"]) + "-" + str(row[1]["num_mnth"])  
        # Check if key exists in availability dictionary
        if ym_key in yr_mnth_avl.keys():
            # If exists, then add +1 to availability if count > 0
            if (row[1][col] > 0):
                yr_mnth_avl[ym_key] = yr_mnth_avl[ym_key] + 1
        else:
            if (row[1][col] > 0):
                yr_mnth_avl[ym_key] = 1
    # Filter data on values more than zero
    cnt_dat_avl = len(dat_avl[dat_avl[col] > 0])
    if (cnt_dat_avl > 2):
        cols_gt_2.append(col)
    # Print availability
    print("INFO: Column - " + col + ". Availability (Months) - " + str(cnt_dat_avl))

INFO: Column - age. Availability (Months) - 4
INFO: Column - gender. Availability (Months) - 2
INFO: Column - tall_cm. Availability (Months) - 3
INFO: Column - weight_kg. Availability (Months) - 3
INFO: Column - cigarettesyn. Availability (Months) - 1
INFO: Column - healthconditions___1. Availability (Months) - 3
INFO: Column - healthconditions___2. Availability (Months) - 3
INFO: Column - healthconditions___3. Availability (Months) - 3
INFO: Column - healthconditions___4. Availability (Months) - 3
INFO: Column - healthconditions___5. Availability (Months) - 3
INFO: Column - healthconditions___6. Availability (Months) - 3
INFO: Column - healthconditions___7. Availability (Months) - 3
INFO: Column - healthconditions___8. Availability (Months) - 3
INFO: Column - healthconditions___9. Availability (Months) - 3
INFO: Column - healthconditions___10. Availability (Months) - 3
INFO: Column - healthconditions___11. Availability (Months) - 3
INFO: Column - healthconditions___12. Availability (M

In [40]:
# Based on the above analysis, identify the list of columns relating to demographic health columns
 # Listing columns that are focused on health demographics
# Includes - Age, Gender, Height, Weight, Health Conditions, Hours of Sleep, Quality, Trouble
dmg_hlth_cols_avl = [
    "age",
    "tall_cm",
    "weight_kg",
    "healthconditions___1",
    "healthconditions___2",
    "healthconditions___3",
    "healthconditions___4",
    "healthconditions___5",
    "healthconditions___6",
    "healthconditions___7",
    "healthconditions___8",
    "healthconditions___9",
    "healthconditions___10",
    "healthconditions___11",
    "healthconditions___12",
    "healthconditions___13",
    "healthconditions___14",
    "healthconditions___15",
    "healthconditions___16",
    "healthconditions___17",
    "healthconditions___18",
    "working___1",
    "working___2",
    "working___3",
    "working___4",
    "working___5",
    "working___6",
    "working___7",
    "working___8",
    "location",
    "householdnumber",
    "householdcomp___1",
    "householdcomp___2",
    "householdcomp___3",
    "householdcomp___4",
    "householdcomp___5",
    "householdcomp___6",
    "householdcomp___7",
    "hoursofsleepweekdays",
    "hoursofsleepweekends",
    "timeoutside",
    "readingtalking"
]

In [41]:
# Filter data to exclude month of May 2020
crisis_pa_adlt_h_f = crisis_pa_adlt[~((crisis_pa_adlt["num_mnth"]==5) & (crisis_pa_adlt["num_yr"]==2020))].copy(True)

In [42]:
# Imputation for age, tall and weight
crisis_pa_adlt_h_f["age_imp"] = crisis_pa_adlt_h_f["age"]
crisis_pa_adlt_h_f["tall_cm_imp"] = crisis_pa_adlt_h_f["tall_cm"]
crisis_pa_adlt_h_f["weight_kg_imp"] = crisis_pa_adlt_h_f["weight_kg"]
# Medians
age_median = crisis_pa_adlt_h_f["age"].median()
tall_cm_median = crisis_pa_adlt_h_f["tall_cm"].median()
weight_kg_median = crisis_pa_adlt_h_f["weight_kg"].median()
# Replace with median
crisis_pa_adlt_h_f["age_imp"] = crisis_pa_adlt_h_f["age"].apply(lambda r: age_median if numpy.isnan(r) else r)
crisis_pa_adlt_h_f["tall_cm_imp"] = crisis_pa_adlt_h_f["tall_cm"].apply(lambda r: tall_cm_median if numpy.isnan(r) else r)
crisis_pa_adlt_h_f["weight_kg_imp"] = crisis_pa_adlt_h_f["weight_kg"].apply(lambda r: weight_kg_median if numpy.isnan(r) else r)

In [335]:
# Gender analysis is dropped as it was not present in multiple months.
# Gender dictionary for imputation
# 1, Man | 2, Woman | 3, Non-binary | 4, Transgender man | 5, Transgender woman | 6, Other
# crisis_pa_adlt["gender_m"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r==1 else 0)
# crisis_pa_adlt["gender_f"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r==2 else 0)
# crisis_pa_adlt["gender_o"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r > 2 else 0)
# Unknown
# crisis_pa_adlt["gender_u"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if numpy.isnan(r) else 0)

In [43]:
crisis_pa_adlt_h_f["hoursofsleepweekdays_imp"] = crisis_pa_adlt_h_f["hoursofsleepweekdays"]
crisis_pa_adlt_h_f["hoursofsleepweekends_imp"] = crisis_pa_adlt_h_f["hoursofsleepweekends"]
hoswd_median = crisis_pa_adlt_h_f["hoursofsleepweekdays"].median()
hoswe_median = crisis_pa_adlt_h_f["hoursofsleepweekends"].median()
crisis_pa_adlt_h_f["hoursofsleepweekdays_imp"] = crisis_pa_adlt_h_f["hoursofsleepweekdays"].apply(lambda r: hoswd_median if numpy.isnan(r) else r)
crisis_pa_adlt_h_f["hoursofsleepweekends_imp"] = crisis_pa_adlt_h_f["hoursofsleepweekends"].apply(lambda r: hoswe_median if numpy.isnan(r) else r)

In [44]:
# Imputing the health conditions columns
for k in range(1, 19):
    crisis_pa_adlt_h_f["healthconditions___" + str(k) + "_imp"] = crisis_pa_adlt_h_f["healthconditions___" + str(k)].apply(lambda r: 0 if numpy.isnan(r) else r)

In [45]:
# Imputing the working columns
for k in range(1, 9):
    crisis_pa_adlt_h_f["working___" + str(k) + "_imp"] = crisis_pa_adlt_h_f["working___" + str(k)].apply(lambda r: 0 if numpy.isnan(r) else r)

In [46]:
# Imputing the household comp columns
for k in range(1, 8):
    crisis_pa_adlt_h_f["householdcomp___" + str(k) + "_imp"] = crisis_pa_adlt_h_f["householdcomp___" + str(k)].apply(lambda r: 0 if numpy.isnan(r) else r)

In [47]:
# Imputing the household number column
# We provide 0 as default in case number of household members not provided
crisis_pa_adlt_h_f["householdnumber_imp"] = crisis_pa_adlt_h_f["householdnumber"].apply(lambda r: 0 if numpy.isnan(r) else r)

In [48]:
# Imputing location based on median value
crisis_pa_adlt_h_f["location_imp"] = crisis_pa_adlt_h_f["location"]
location_median = crisis_pa_adlt_h_f["location"].median()
crisis_pa_adlt_h_f["location_imp"] = crisis_pa_adlt_h_f["location"].apply(lambda r: location_median if numpy.isnan(r) else r)

In [49]:
# Imputing time outside and reading talking
crisis_pa_adlt_h_f["timeoutside_imp"] = crisis_pa_adlt_h_f["timeoutside"].apply(lambda r: 1 if numpy.isnan(r) else r)
crisis_pa_adlt_h_f["readingtalking_imp"] = crisis_pa_adlt_h_f["readingtalking"].apply(lambda r: 1 if numpy.isnan(r) else r)

In [50]:
# Filter data for cases where mental health (worried) is available.
crisis_pa_adlt_h_f_wr = crisis_pa_adlt_h_f.copy(True)
crisis_pa_adlt_h_f_wr["max_worried"] = crisis_pa_adlt_h_f_wr["max_worried"].apply(lambda r: 1 if numpy.isnan(r) else r)
crisis_pa_adlt_h_f_wr["max_worried"] = crisis_pa_adlt_h_f_wr["max_worried"].astype(int)
crisis_pa_adlt_h_f_wr["max_worried_1"] = crisis_pa_adlt_h_f_wr["max_worried"].apply(lambda r: 0 if r in (1, 2, 3) else 1)

In [51]:
for v in crisis_pa_adlt_h_f_wr["max_worried_1"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_h_f_wr = crisis_pa_adlt_h_f_wr.copy(True)
    crisis_pa_adlt_h_f_wr["wr_tgt"] = crisis_pa_adlt_h_f_wr["max_worried_1"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Max worried state - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_h_f_wr[[col + "_imp" for col in dmg_hlth_cols_avl]], crisis_pa_adlt_h_f_wr["wr_tgt"], test_size=0.25, random_state=300)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000, max_depth=5, min_samples_leaf=100)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Max worried state - 0
INFO: Training Metrics
[[1704  763]
 [ 529 2416]]
INFO: Testing metrics
[[482 348]
 [275 699]]
0.6491910145716335

INFO: Training model for Max worried state - 1
INFO: Training Metrics
[[2416  529]
 [ 763 1704]]
INFO: Testing metrics
[[699 275]
 [348 482]]
0.6491910145716335



In [52]:
# Filter data for cases where mental health (worried) is available.
crisis_pa_adlt_h_f_wr = crisis_pa_adlt_h_f.copy(True)
crisis_pa_adlt_h_f_wr["worriedmental"] = crisis_pa_adlt_h_f_wr["worriedmental"].apply(lambda r: 1 if numpy.isnan(r) else r)
crisis_pa_adlt_h_f_wr["worriedmental"] = crisis_pa_adlt_h_f_wr["worriedmental"].astype(int)
crisis_pa_adlt_h_f_wr["worriedmental_1"] = crisis_pa_adlt_h_f_wr["worriedmental"].apply(lambda r: 0 if r in (1, 2, 3) else 1)

In [53]:
for v in crisis_pa_adlt_h_f_wr["worriedmental_1"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_h_f_wr = crisis_pa_adlt_h_f_wr.copy(True)
    crisis_pa_adlt_h_f_wr["wr_tgt"] = crisis_pa_adlt_h_f_wr["worriedmental_1"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Max worried state - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_h_f_wr[[col + "_imp" for col in dmg_hlth_cols_avl]], crisis_pa_adlt_h_f_wr["wr_tgt"], test_size=0.2, random_state=240)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1200, max_depth=5, min_samples_leaf=100)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Max worried state - 0
INFO: Training Metrics
[[1016  481]
 [  80 4195]]
INFO: Testing metrics
[[125 226]
 [104 989]]
0.6304871977333094

INFO: Training model for Max worried state - 1
INFO: Training Metrics
[[4195   80]
 [ 481 1016]]
INFO: Testing metrics
[[989 104]
 [226 125]]
0.6304871977333093



In [54]:
# Correlations
crisis_pa_adlt_h_f_wr[[col + "_imp" for col in dmg_hlth_cols_avl] + ["worriedmental"]].corr()

,age_imp,tall_cm_imp,weight_kg_imp,healthconditions___1_imp,healthconditions___2_imp,healthconditions___3_imp,healthconditions___4_imp,healthconditions___5_imp,healthconditions___6_imp,healthconditions___7_imp,...,householdcomp___3_imp,householdcomp___4_imp,householdcomp___5_imp,householdcomp___6_imp,householdcomp___7_imp,hoursofsleepweekdays_imp,hoursofsleepweekends_imp,timeoutside_imp,readingtalking_imp,worriedmental
age_imp,1.000000,0.091015,0.170686,-0.024300,-0.036858,0.258185,0.067966,0.033105,0.212960,0.209606,...,-0.080708,-0.329195,0.135066,-0.028513,-0.128247,-0.124195,-0.240073,0.086471,0.074850,-0.224976
tall_cm_imp,0.091015,1.000000,0.319673,-0.045541,-0.026467,0.042337,-0.022133,-0.073769,0.031159,-0.008387,...,-0.024311,0.015488,-0.034142,-0.037921,0.001718,-0.024770,-0.051697,0.071020,-0.007540,-0.095160
weight_kg_imp,0.170686,0.319673,1.000000,-0.006206,0.032520,0.087233,-0.004751,0.008637,0.143806,0.015565,...,-0.038933,-0.054303,0.048302,0.021065,-0.045932,-0.080435,-0.080204,-0.029003,0.010075,-0.015940
healthconditions___1_imp,-0.024300,-0.045541,-0.006206,1.000000,0.211901,0.031981,0.018663,0.066724,-0.013873,0.010990,...,0.010067,0.008797,-0.021719,-0.025679,0.023838,-0.022266,0.032564,-0.034979,0.063106,0.082794
healthconditions___2_imp,-0.036858,-0.026467,0.032520,0.211901,1.000000,0.049333,0.043494,0.030788,0.021475,0.023993,...,0.018970,0.030033,0.002862,0.016724,0.002228,-0.014202,0.010387,-0.000736,-0.008196,0.049299
healthconditions___3_imp,0.258185,0.042337,0.087233,0.031981,0.049333,1.000000,0.120059,0.036002,0.168333,0.085274,...,-0.007419,-0.059318,-0.009393,-0.034746,-0.016848,-0.018873,-0.057893,-0.000297,-0.005910,-0.046976
healthconditions___4_imp,0.067966,-0.022133,-0.004751,0.018663,0.043494,0.120059,1.000000,0.064986,0.095553,0.059397,...,0.007112,-0.021985,0.007641,-0.019310,0.003859,-0.014751,-0.022487,-0.007982,-0.000475,0.005966
healthconditions___5_imp,0.033105,-0.073769,0.008637,0.066724,0.030788,0.036002,0.064986,1.000000,0.006436,0.048819,...,0.014103,-0.009554,0.029032,-0.003881,0.007480,-0.001097,-0.006993,-0.033271,0.036024,0.041755
healthconditions___6_imp,0.212960,0.031159,0.143806,-0.013873,0.021475,0.168333,0.095553,0.006436,1.000000,0.053623,...,-0.025571,-0.040339,0.034773,-0.001052,-0.018974,-0.049246,-0.070219,-0.040554,-0.002765,-0.051866
healthconditions___7_imp,0.209606,-0.008387,0.015565,0.010990,0.023993,0.085274,0.059397,0.048819,0.053623,1.000000,...,-0.021494,-0.049418,-0.018548,-0.003541,-0.022247,-0.015458,-0.038753,0.002429,0.048971,-0.016415


#### Concluding on using health based demographic features to predict whether an individual would be worried or not
#### On creating a binary variable for "MAX WORRIED", i.e. the individual is likely to be worried themselves or worry others, the classifier is weak at best
#### We created an GB model with AUC ~ 0.65 which is weak in nature
#### Key contributing factors towards predicting worry - Age, weight, household

#### For the outcome - Mental Worry, we received similar results


In [80]:
# Filter data for cases where mental health (worried) is available.
crisis_pa_adlt_h_f_im = crisis_pa_adlt_h_f.copy(True)
for k in range(1, 8):
     crisis_pa_adlt_h_f_im["impact___" + str(k) + "_imp"] = crisis_pa_adlt_h_f_im["impact___" + str(k)].apply(lambda r: 0 if numpy.isnan(r) else r)

# Impact 8 receives separate treatment as it is None of the above / No impact. So cases where it is not present, we make it as 1.
crisis_pa_adlt_h_f_im["impact___8" + "_imp"] = crisis_pa_adlt_h_f_im["impact___8"].apply(lambda r: 1 if numpy.isnan(r) else r)

#### Creating a model to predict / determine impact of health, demographic and substance related columns to the overall impact
#### Impact has been vectorized into 8 columns - impact__1 to 8
#### We first impute the impact to - 0 (cases where it is not provided). Assumption that it would be "no impact in that field"
#### On training a baseline Model using Gradient Boosting, it does not provide any lift
#### The AUC is between 0.53 to 0.6, which is very weak in terms of determining outcome (impact___*) from features
#### Feature used include - Health based features available across 3 months and substance based features
#### Key features for Impact are - 2               
##### weight_kg_imp, age_imp, tall_cm_imp, location_imp, timeoutside_imp, householdnumber_imp, alcohol_imp, readingtalking_imp
#### Note that we imputed age, weight etc. with median. One possibility is try determining impact without any imputation and only for the section of population that have provided details w.r.t it


In [81]:
for v in range(1, 9):
    # Filter data for each mental health value.
    crisis_pa_adlt_h_f_im = crisis_pa_adlt_h_f_im.copy(True)
    crisis_pa_adlt_h_f_im["im_tgt"] = crisis_pa_adlt_h_f_im["impact___" + str(v) + "_imp"]
    # Print information
    print("INFO: Training model for Impact - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_h_f_im[[col + "_imp" for col in dmg_hlth_cols_avl]], crisis_pa_adlt_h_f_im["im_tgt"], test_size=0.2, random_state=350)
    # Train a Gradient boosted classifier
    gb_im = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1250, max_depth=8, min_samples_leaf=30)
    gb_im = gb_im.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_im.predict(X_train)
    y_pred_test = gb_im.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Impact - 1
INFO: Training Metrics
[[5370    0]
 [   0  402]]
INFO: Testing metrics
[[1326   12]
 [  98    8]]
0.5332515441238683

INFO: Training model for Impact - 2
INFO: Training Metrics
[[5579    0]
 [   0  193]]
INFO: Testing metrics
[[1389    3]
 [  49    3]]
0.5277685676392573

INFO: Training model for Impact - 3
INFO: Training Metrics
[[4949    0]
 [   0  823]]
INFO: Testing metrics
[[1183   61]
 [ 176   24]]
0.5354823151125402

INFO: Training model for Impact - 4
INFO: Training Metrics
[[4902    0]
 [   0  870]]
INFO: Testing metrics
[[1179   61]
 [ 186   18]]
0.5195208728652752

INFO: Training model for Impact - 5
INFO: Training Metrics
[[5150    0]
 [   0  622]]
INFO: Testing metrics
[[1264   41]
 [ 125   14]]
0.5346508999696795

INFO: Training model for Impact - 6
INFO: Training Metrics
[[4631    0]
 [   0 1141]]
INFO: Testing metrics
[[1061   94]
 [ 249   40]]
0.5285115115564943

INFO: Training model for Impact - 7
INFO: Training Metrics
[[5622    0

In [96]:
for v in range(1, 9):
    # Filter data for each mental health value.
    crisis_pa_adlt_h_f_im = crisis_pa_adlt_h_f_im.copy(True)
    crisis_pa_adlt_h_f_im["im_tgt"] = crisis_pa_adlt_h_f_im["impact___" + str(v) + "_imp"]
    # Print information
    print("INFO: Training model for Impact - " + str(v))
    # Removing cols based on feature importance (low)
    rm_cols = ["rxsleeping_imp", "rxstimulants_imp", "heroin_imp", "healthconditions___16_imp", "healthconditions___17_imp", "healthconditions___17_imp", "healthconditions___10_imp", "rxother_imp"]
    cols = [col + "_imp" for col in dmg_hlth_cols_avl] + [col + "_imp" for col in dmg_subs_cols_imp_1]
    # Final columns = cols - rm_cols
    fcols = []
    for c in cols:
        if c in rm_cols:
            continue
        else:
            fcols.append(c)
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_h_f_im[fcols], crisis_pa_adlt_h_f_im["im_tgt"], test_size=0.2, random_state=360)
    # Train a Gradient boosted classifier
    gb_im = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1250, max_depth=6, min_samples_leaf=15)
    gb_im = gb_im.fit(X_train, y_train)
    print(pandas.DataFrame(list(zip(gb_im.feature_names_in_, gb_im.feature_importances_))).sort_values(1, ascending=False))
    # Predict on training and test
    y_pred_train = gb_im.predict(X_train)
    y_pred_test = gb_im.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Impact - 1
                            0         1
2               weight_kg_imp  0.183062
0                     age_imp  0.151752
1                 tall_cm_imp  0.122793
39                alcohol_imp  0.050092
27        householdnumber_imp  0.046130
26               location_imp  0.043069
37            timeoutside_imp  0.041909
38         readingtalking_imp  0.040479
18            working___1_imp  0.022885
35   hoursofsleepweekdays_imp  0.022276
36   hoursofsleepweekends_imp  0.022213
14  healthconditions___13_imp  0.021530
43              marijuana_imp  0.018555
40             cigarettes_imp  0.017760
3    healthconditions___1_imp  0.017104
4    healthconditions___2_imp  0.013892
28      householdcomp___1_imp  0.013730
32      householdcomp___5_imp  0.012419
13  healthconditions___12_imp  0.011279
19            working___2_imp  0.010888
41                   vape_imp  0.009089
11   healthconditions___9_imp  0.007613
10   healthconditions___8_imp  0.007323
29  

In [77]:
crisis_pa_adlt_h_f_im.groupby(["impact___8"]).count()

,ID,timestamp1,country,age,sex,sex_other,raceethnicity___1,raceethnicity___2,raceethnicity___3,raceethnicity___4,...,location_imp,timeoutside_imp,readingtalking_imp,impact___1_imp,impact___2_imp,impact___3_imp,impact___4_imp,impact___5_imp,impact___6_imp,impact___7_imp
impact___8,,,,,,,,,,,,,,,,,,,,,
0,3036,3035,3033,3031,1160,4,2078,2078,2078,2078,...,3036,3036,3036,3036,3036,3036,3036,3036,3036,3036
1,4180,4180,4179,4172,1679,7,3013,3013,3013,3013,...,4180,4180,4180,4180,4180,4180,4180,4180,4180,4180
